# 🏠 DeepRoof-2026: Multi-Task Training Notebook

### 🛠 Step 1: Nuclear Environment Repair
This cell is designed to fix **container-specific** pathING and library issues (missing CUDA libs, venv activation).

In [ ]:
import os
import sys
import subprocess
import torch
import ctypes
import glob
from pathlib import Path

def nuclear_fix():
    print("🛰 Starting Nuclear Environment Repair...")
    
    # --- 1. FIND PROJECT ROOT & VENV ---
    # Traceback says we are in /workspace/roof/notebooks
    possible_roots = ["/workspace/roof", str(Path(os.getcwd()).parent)]
    project_root = None
    for pr in possible_roots:
        if os.path.exists(pr):
            project_root = Path(pr)
            break
    
    if not project_root:
        print("❌ Could not find project root. Manual pathing required.")
        return False
    
    print(f"📂 Root Detected: {project_root}")
    
    # Find ANY venv directory
    venv_names = [".venv", "venv", "env"]
    found_venv = None
    for vn in venv_names:
        vp = project_root / vn
        if vp.exists():
            found_venv = vp
            break
            
    if found_venv:
        print(f"🐍 Venv Found: {found_venv}")
        lib_dirs = list(found_venv.glob("lib/python*/site-packages"))
        if lib_dirs:
            venv_site = str(lib_dirs[0])
            if venv_site not in sys.path:
                sys.path.insert(0, venv_site)
                print(f"✅ Prioritized Venv Site: {venv_site}")
                # Redirect pip calls to the venv
                sys.executable = str(found_venv / "bin" / "python")
    else:
        print("⚠️ No venv found in root. Using system environment.")

    if str(project_root) not in sys.path:
        sys.path.insert(0, str(project_root))
        
    # --- 2. FIX CUDA LINKING (libcudart.so) ---
    cuda_available = torch.cuda.is_available()
    print(f"🚀 CUDA Ready: {cuda_available}")
    
    if cuda_available:
        print("🔍 Searching for CUDA libraries in container...")
        # Check common system locations for A100 containers
        search_paths = [
            "/usr/local/cuda/lib64/libcudart.so*",
            "/usr/lib/x86_64-linux-gnu/libcudart.so*",
            "/usr/local/lib/python*/dist-packages/nvidia/cuda_runtime/lib/libcudart.so*"
        ]
        
        found_lib = None
        for sp in search_paths:
            matches = glob.glob(sp)
            if matches:
                found_lib = matches[0]
                break
        
        if found_lib:
            print(f"📍 Found linking target: {found_lib}")
            try:
                ctypes.CDLL(found_lib, mode=ctypes.RTLD_GLOBAL)
                print("✅ Forced libcudart into memory.")
            except Exception as e:
                print(f"❌ Failed to load {found_lib}: {e}")
        else:
            print("⚠️ libcudart not found on system. Installing runtime package...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", "nvidia-cuda-runtime-cu11"])
            return False # Restart needed

    # --- 3. REPAIR MMSEGMENTATION --- 
    try: 
        import mmseg
        init_file = Path(mmseg.__file__).parent / "__init__.py"
        with open(init_file, 'r') as f: content = f.read()
        if "OVERRIDE by DeepRoof" not in content:
            print(f"🩹 Patching mmseg init in current context: {init_file}")
            with open(init_file, 'w') as f:
                f.write("""# Copyright (c) OpenMMLab. All rights reserved.
import mmcv
import mmengine
from mmengine.utils import digit_version
from .version import __version__, version_info
MMCV_MIN = '2.0.0rc4'
MMCV_MAX = '2.2.0'
MMENGINE_MIN = '0.7.1'
MMENGINE_MAX = '1.0.0'
mmcv_min_version = digit_version(MMCV_MIN)
mmcv_max_version = digit_version('9.9.9') # OVERRIDE by DeepRoof
mmcv_version = digit_version(mmcv.__version__)
mmengine_min_version = digit_version(MMENGINE_MIN)
mmengine_max_version = digit_version('9.9.9') # OVERRIDE by DeepRoof
mmengine_version = digit_version(mmengine.__version__)
__all__ = ['__version__', 'version_info', 'digit_version']\n""")
            return False
    except: pass

    # --- 4. MMCV BINARY CHECK ---
    try:
        import mmcv
        from mmcv.ops import point_sample
        has_ops = True
    except: has_ops = False
    
    if cuda_available and not has_ops:
        print("🔄 Installing CUDA-enabled MMCV...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "mmcv==2.2.0", "-f", "https://download.openmmlab.com/mmcv/dist/cu118/torch2.1/index.html"])
        return False
    
    print("✅ Environment Ready.")
    return True

if nuclear_fix():
    print(f"🚀 System Live | {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU Mode'}")

## 📂 1. Dataset Preview

Visualize the **satellite imagery**, **instance masks**, and **surface normals**.

In [ ]:
def preview_dataset(data_root, num_samples=3):
    import matplotlib.pyplot as plt
    import numpy as np
    import cv2
    
    data_path = Path(data_root)
    # Resolve relative to project root found in Cell 1
    try:
        import __main__
        root = Path(__main__.project_root)
    except: root = Path(os.getcwd()).parent
    
    if not data_path.is_absolute():
        data_path = root / data_root
        
    train_file = data_path / 'train.txt'
    if not train_file.exists():
        print(f"❌ Could not find train.txt at {train_file}. Run prepare_omnicity_v2_final.py first!")
        return
        
    with open(train_file, 'r') as f:
        sample_ids = [line.strip() for line in f.readlines()[:num_samples]]
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5 * num_samples))
    for i, sid in enumerate(sample_ids):
        img_path = str(data_path / 'images' / (sid + '.jpg'))
        img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        
        mask_path = str(data_path / 'masks' / (sid + '.png'))
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        mask_vis = cv2.applyColorMap(((mask % 20) * 12).astype(np.uint8), cv2.COLORMAP_JET)
        
        axes[i, 0].imshow(img); axes[i, 0].set_title(sid); axes[i, 0].axis('off')
        axes[i, 1].imshow(mask_vis); axes[i, 1].set_title("Mask"); axes[i, 1].axis('off')
        
        norm_path = data_path / 'normals' / (sid + '.npy')
        if norm_path.exists():
            normals = np.load(str(norm_path))
            axes[i, 2].imshow(((normals + 1) * 127.5).astype(np.uint8))
        axes[i, 2].set_title("Normals"); axes[i, 2].axis('off')
        
    plt.tight_layout(); plt.show()

preview_dataset("data/OmniCity", num_samples=2)

## ⚙️ 2. Training Configuration


In [ ]:
from mmengine.config import Config

MODE = "fine-tune" 
try:
    import __main__
    root = Path(__main__.project_root)
except: root = Path(os.getcwd()).parent

CONFIG_FILE = str(root / "configs/deeproof_finetune_swin_L.py")
WORK_DIR = str(root / "work_dirs/swin_l_omnicity_v2")

cfg = Config.fromfile(CONFIG_FILE)
cfg.work_dir = WORK_DIR
cfg.data_root = str(root / "data/OmniCity/")
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.train_cfg.max_iters = 20000

if MODE == "scratch": cfg.load_from = None
print(f"✅ Configuration Validated. WorkDir: {WORK_DIR}")

## 🚀 3. Start Training


In [ ]:
import torch
from mmengine.runner import Runner

print(f"🚀 Starting Trainer on: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

runner = Runner.from_cfg(cfg)
runner.train()